In [ ]:
CMLS 2023 - Fab Four - Group ID: 6 - Python implementation of a continuous blendable multimode filter

In [ ]:
import math

class Filter:
    def __init__(self):
        self.self_osc_push = False
        self.band_pass_sw = False
        self.mm = 0
        self.s1 = self.s2 = self.s3 = self.s4 = 0
        self.sample_rate = 44100
        self.sample_rate_inv = 1 / self.sample_rate
        rcrate = math.sqrt((44100 / self.sample_rate))
        self.rcor = (500.0 / 44100) * rcrate
        self.rcor24 = (970.0 / 44100) * rcrate
        self.rcor_inv = 1 / self.rcor
        self.rcor24_inv = 1 / self.rcor24
        self.R = 1
        self.R24 = 0

    def set_multimode(self, m):
        self.mm = m
        self.mmch = int(self.mm * 3)
        self.mmt = self.mm * 3 - self.mmch

    def set_sample_rate(self, sr):
        self.sample_rate = sr
        self.sample_rate_inv = 1 / self.sample_rate
        rcrate = math.sqrt((44100 / self.sample_rate))
        self.rcor = (500.0 / 44100) * rcrate
        self.rcor24 = (970.0 / 44100) * rcrate
        self.rcor_inv = 1 / self.rcor
        self.rcor24_inv = 1 / self.rcor24

    def set_resonance(self, res):
        self.R = 1 - res
        self.R24 = 3.5 * res

    def diode_pair_resistance_approx(self, x):
        # Taylor approximation of slightly mismatched diode pair
        return (((((0.0103592 * x + 0.00920833) * x + 0.185) * x + 0.05) * x + 1.0))

    def nr(self, sample, g):
        # Calculating feedback non-linear transconductance and compensated for R (-1)
        # Boosting non-linearity
        if not self.self_osc_push:
            t_cfb = self.diode_pair_resistance_approx(self.s1 * 0.0876) - 1.0
        else:
            t_cfb = self.diode_pair_resistance_approx(self.s1 * 0.0876) - 1.035

        # Resolve linear feedback
        y = ((sample - 2 * (self.s1 * (self.R + t_cfb)) - g * self.s1 - self.s2) / (1 + g * (2 * (self.R + t_cfb) + g)))

        return y

    def apply(self, sample, g):
        gpw = math.tan(g * self.sample_rate_inv * math.pi)
        g = gpw
        v = self.nr(sample, g)

        y1 = v * g + self.s1
        self.s1 = v * g + y1

        y2 = y1 * g + self.s2
        self.s2 = y1 * g + y2

        if not self.band_pass_sw:
            mc = (1 - self.mm) * y2 + (self.mm) * v
        else:
            if self.mm < 0.5:
                mc = 2 * ((0.5 - self.mm) * y2 + (self.mm) * y1)
            else:
                mc = 2 * ((1 - self.mm) * y1 + (self.mm - 0.5) * v)
        return mc

    def nr24(self, sample, g, lpc):
        ml = 1 / (1 + g)
        S = (lpc * (lpc * (lpc * self.s1 + self.s2) + self.s3) + self.s4) * ml
        G = lpc * lpc * lpc * lpc
        y = (sample - self.R24 * S) / (1 + self.R24 * G)
        return y

    def apply4pole(self, sample, g):
        g1 = math.tan(g * self.sample_rate_inv * math.pi)
        g = g1

        lpc = g / (1 + g)
        y0 = self.nr24(sample, g, lpc)
        v = (y0 - self.s1) * lpc
        res = v + self.s1
        self.s1 = res + v
        self.s1 = math.atan(self.s1 * self.rcor24) * self.rcor24_inv

        y1 = res
        y2 = (y1 - self.s2) * lpc
        self.s2 = y1 * lpc + self.s2

        y3 = (y2 - self.s3) * lpc
        self.s3 = y2 * lpc + self.s3

        y4 = (y3 - self.s4) * lpc
        self.s4 = y3 * lpc + self.s4

        if self.mmch == 0:
            mc = ((1 - self.mmt) * y4 + (self.mmt) * y3)
        elif self.mmch == 1:
            mc = ((1 - self.mmt) * y3 + (self.mmt) * y2)
        elif self.mmch == 2:
            mc = ((1 - self.mmt) * y2 + (self.mmt) * y1)
        elif self.mmch == 3:
            mc = y1
        else:
            mc = 0

        # Half volume compensation
        return mc * (1 + self.R24 * 0.45)

